In [7]:
from utils import cls, draw_matrix
from tictactoe.tictactoe import TicTacToe

_size = 10
game = TicTacToe(n=_size)
state = game.initial_state

valid_moves = _size * _size
won = False
current_player = 0
while not won and valid_moves > 0:
  print("won", won)
  row = input(f"Enter row for player {current_player}: ")
  column = input(f"Enter column for player {current_player}: ")
  cls()
  _move = int(row) * _size + int(column)
  state, won = game.move(state, _move, current_player)
  draw_matrix(game.convert_mcts_state_to_list_state(state))
  if won:
    print(f"Player {current_player} won!")
  valid_moves = len(game.possible_moves(state))
  current_player = 1 - current_player

if valid_moves == 0:
  print("It's a draw")


--------------------------------
0 0 2 0 2 2 2 2 2 2 
2 1 2 2 2 2 2 2 2 2 
2 2 1 2 2 2 2 2 2 2 
2 2 2 1 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
2 2 2 2 2 2 2 2 2 2 
--------------------------------
Player 1 won!


In [ ]:
import numpy as np
import torch
from utils import cls, draw_matrix
from model import Net
from mcts.mcts import MCTS
from tictactoe.tictactoe import TicTacToe
import config as cfg

_size = 3
game = TicTacToe(n=_size)
mcts_store = MCTS(game)
state = game.initial_state

model = Net(input_shape=game.obs_shape, actions_n=game.action_space)
model.load_state_dict(torch.load(
            'trained/trained_tictactoe/best_005_00900.dat', map_location=lambda storage, loc: storage))

won = False
valid_moves = _size * _size

while not won and valid_moves > 0:
  row = input("Enter row: ")
  column = input("Enter column: ")
  cls()
  _move = int(row) * _size + int(column)
  state, won = game.move(state, _move, 0)
  draw_matrix(game.convert_mcts_state_to_list_state(state))
  if won:
    print("You won!")
  valid_moves = len(game.possible_moves(state))
  if valid_moves == 0:
    break

  mcts_store.search_batch(
              cfg.BOT_MCTS_SEARCHES, cfg.BOT_MCTS_BATCH_SIZE, state, 1, model)
  probs, values = mcts_store.get_policy_value(state, tau=0)
  action = np.random.choice(game.action_space, p=probs)
  state, won = game.move(state, action, 1)
  draw_matrix(game.convert_mcts_state_to_list_state(state))
  if won:
    print("You lose!!!")
  valid_moves = len(game.possible_moves(state))
if valid_moves == 0:
  print("It's a draw")
